In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\bisht\\OneDrive\\Desktop\\geoguess'

In [4]:
from AI_Vs_Human_Geoguess.constants import *
from AI_Vs_Human_Geoguess.utils.common import read_yaml,create_directories

In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir:Path
    base_model_path:Path
    updated_base_model_path:Path
    params_image_size:list
    params_learning_rate:float
    params_weights:str
    params_output_nueron:int
    params_include_top:bool
    
    


In [6]:
from AI_Vs_Human_Geoguess.constants import *
from AI_Vs_Human_Geoguess.utils.common import read_yaml,create_directories

In [7]:


    
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_weights=self.params.WEIGHTS,
            params_output_nueron=self.params.OUTPUT_NUERON,
            params_include_top=self.params.INCLUDE_TOP
            
        )
        return prepare_base_model_config


In [8]:
import os
import zipfile
import gdown
from AI_Vs_Human_Geoguess import logger
from AI_Vs_Human_Geoguess.utils.common import get_size
import pandas as pd
import tensorflow as tf


In [13]:
class PrepareBaseModel:
    def __init__(self,config=PrepareBaseModelConfig):
        self.config=config
    def get_base_model(self):
        self.model=tf.keras.applications.vgg16.VGG16(
            weights=self.config.params_weights,
            include_top=self.config.params_include_top,
            input_shape=self.config.params_image_size
        )
        self.save_model(path=self.config.base_model_path,model=self.model)
    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)
    @staticmethod
    def  _prepare_full_model(model,output_nueron,freeze_all,freeze_till):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till>0):
            for layer in model.layers[:freeze_all]:
                model.trainable=False
        flatten_in=tf.keras.layers.Flatten()(model.output)
        Dense1 = tf.keras.layers.Dense(512, activation='relu')(flatten_in)
        normalize1 = tf.keras.layers.BatchNormalization()(Dense1)
        dropout1 = tf.keras.layers.Dropout(0.5)(normalize1)
        Dense2 = tf.keras.layers.Dense(256, activation='relu')(dropout1)
        normalize2 = tf.keras.layers.BatchNormalization()(Dense2)
        dropout2 = tf.keras.layers.Dropout(0.4)(normalize2)
        Dense3 = tf.keras.layers.Dense(128, activation='relu')(dropout2)
        normalize3 = tf.keras.layers.BatchNormalization()(Dense3)
        dropout3 = tf.keras.layers.Dropout(0.4)(normalize3)
        Dense4 = tf.keras.layers.Dense(64, activation='relu')(dropout3)
        normalize4 = tf.keras.layers.BatchNormalization()(Dense4)
        dropout4 = tf.keras.layers.Dropout(0.3)(normalize4)
        Dense5 = tf.keras.layers.Dense(32,activation='relu')(dropout4)
        normalize5 = tf.keras.layers.BatchNormalization()(Dense5)
        dropout5 = tf.keras.layers.Dropout(0.3)(normalize5)
        Dense6 = tf.keras.layers.Dense(16, activation='relu')(dropout5)
        normalize6 = tf.keras.layers.BatchNormalization()(Dense6)
        dropout6 = tf.keras.layers.Dropout(0.2)(normalize6)
        outputs = tf.keras.layers.Dense(units=output_nueron, activation='sigmoid')(dropout6)
        full_model=tf.keras.models.Model(
            inputs=model.input,
            outputs=outputs
        )
        full_model.compile(optimizer='adam',loss='mse')
        full_model.summary()
        return full_model
    def update_base_model(self):
        self.full_model=self._prepare_full_model(
            model=self.model,
            output_nueron=self.config.params_output_nueron,
            freeze_all=True,
            freeze_till=None
            
        )
        self.save_model(path=self.config.updated_base_model_path,model=self.full_model)



In [14]:
try:
    config=ConfigurationManager()
    prepare_base_model_config=config.get_prepare_base_model_config()
    prepare_base_model=PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-08-04 21:52:46,882: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-04 21:52:46,885: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-04 21:52:46,886: INFO: common: created directory at: artifacts]
[2024-08-04 21:52:46,888: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-08-04 21:52:47,274: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 